# 3.3章　複雑なフローをこなすエージェント

In [2]:
# !pip install langchain
# !pip install langchain-openai

# !pip install serpapi
# !pip install google-search-results

# # load_toolsを利用するのに必要
# !pip install langchain_community

In [3]:
import os
# from google.colab import userdata
from dotenv import load_dotenv
load_dotenv("/home/rmasuda/book-llm-agent/.env")

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = os.getenv('SERPAPI_API_KEY')

# 3.3章

In [4]:
# !pip install langchainhub

In [5]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

print(prompt.template)

/home/rmasuda/.cache/pypoetry/virtualenvs/llm-agent-U4qSWGiP-py3.11/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [6]:
# おみくじツール定義（再掲）

# おみくじ関数

import random
from datetime import datetime

def get_fortune(date_string):
    # 日付文字列を解析
    try:
        date = datetime.strptime(date_string, "%m月%d日")
    except ValueError:
        return "無効な日付形式です。'X月X日'の形式で入力してくださ\
い。"

    # 運勢のリスト
    fortunes = [
        "大吉", "中吉", "小吉", "吉", "末吉", "凶", "大凶"
    ]

    # 運勢の重み付け（大吉と大凶の確率を低くする）
    weights = [1, 3, 3, 4, 3, 2, 1]

    # 日付に基づいてシードを設定（同じ日付なら同じ運勢を返す）
    random.seed(date.month * 100 + date.day)

    # 運勢をランダムに選択
    fortune = random.choices(fortunes, weights=weights)[0]

    return f"{date_string}の運勢は【{fortune}】です。"

# ツール作成

from langchain.tools import BaseTool

class Get_fortune(BaseTool):
    name: str = 'Get_fortune'
    description: str = (
        "特定の日付の運勢を占う。インプットは  'date_string'です。\
'date_string' は、占いを行う日付で、mm月dd日 という形式です。「1月1日」\
のように入力し、「'1月1日'」のように余計な文字列を付けてはいけません。"
    )

    def _run(self, date_string) -> str:
        return get_fortune(date_string)


    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

In [7]:
# 日付ツール定義（再掲）

# 日付取得関数

from datetime import timedelta
from zoneinfo import ZoneInfo


def get_date(date):
    date_now = datetime.now(ZoneInfo("Asia/Tokyo"))
    if ("今日" in date):
        date_delta = 0
    elif ("明日" in date):
        date_delta = 1
    elif ("明後日" in date):
        date_delta = 2
    else:
        return "サポートしていません"
    return (date_now + timedelta(days=date_delta)).strftime\
('%m月%d日')

class Get_date(BaseTool):
    name: str = 'Get_date'
    description: str = (
        "今日の日付を取得する。インプットは 'date'です。'date' は、日\
付を取得する対象の日で、'今日', '明日', '明後日' という3種類の文字列\
から指定します。「今日」のように入力し、「'今日'」のように余計な文字列を付\
けてはいけません。"
    )

    def _run(self, date) -> str:
        return get_date(date)

    async def _arun(self, query: str) -> str:
        raise NotImplementedError("does not support async")

In [8]:
# エージェントの作成

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.agents import AgentExecutor, create_react_agent

model = ChatOpenAI(model="gpt-4o-mini")

tools = [Get_date(), Get_fortune()]

#1 エージェントの作成
agent = create_react_agent(model, tools, prompt)

#2 エージェントの実行準備
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": [HumanMessage\
(content="今日の運勢を教えてください。")]})

print(response)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I need to find out today's date in order to get the fortune for today. 
Action: Get_date
Action Input: 今日05月01日I now have today's date, which is May 1st. Now, I will use this date to get the fortune for today. 
Action: Get_fortune
Action Input: 5月01日5月01日の運勢は【吉】です。I now know the final answer. 
Final Answer: 今日の運勢は【吉】です。

> Finished chain.
{'input': [HumanMessage(content='今日の運勢を教えてください。')], 'output': '今日の運勢は【吉】です。'}


In [9]:
# /home/rmasuda/.cache/pypoetry/virtualenvs/llm-agent-U4qSWGiP-py3.11/lib/python3.11/site-packages/langchain_community/utilities/serpapi.py
# の67行目を以下に変更する。
from serpapi.google_search import GoogleSearch

In [10]:
# 検索できるエージェント作成

from langchain.agents import load_tools

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


response = agent_executor.invoke({"input": [HumanMessage\
 (content="株式会社Elithの住所を教えてください。最新の公式情報として公\
 開されているものを教えてください。")]})

print(response)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


株式会社Elithの最新の公式情報を確認するために、住所を検索する必要があります。公式ウェブサイトや最新のプレスリリースを探すのが良いでしょう。  
Action: Search  
Action Input: 株式会社Elith 住所 公式サイト  ['株式会社エリスの公式HPです。 Elithは、最先端のAI技術をビジネス実装し価値を生み出す松尾研発スタートアップです.', '\u200bElith OFFICE · 東京都文京区本郷3-30-10本郷 K&Kビル1F · 東京メトロ丸ノ内線 / 本郷三丁目駅 徒歩約2分 都営大江戸線 / 本郷三丁目駅 徒歩約3分 東京メトロ千代田線 / ...', '株式会社Elith_オートクチュールAI開発_パンフレット202409. ソーシャルメディア. 連絡先. https://elith.co.jp contact@elith.co.jp 03-6822-5999. 住所. 日本113-0033 ...', '連絡先. https://www.elith.co.jp/ contact@elith.co.jp. 住所. 日本113-0033東京都文京区本郷3-30-10本郷 K&Kビル1F. この出展社を見ている人は、こちらの出展社も見てい ...', '井上 顧基 氏. 株式会社Elith Founder & CTO. 北陸先端科学技術大学院大学にて量子コンピュータの材料探索の研究で修士号を取得。会社経営と同時に東北 ...', '"AIの「今」をON AIR" 株式会社Elithが主催する、AIの最新動向についてラフに語るラジオです 「AIでこれから世の中はどう変わるのか？」「自分たちはAIをどのように ...', '東大・松尾研発スタートアップの株式会社Elith(エリス)の公式アカウントです。 AI開発・研究を行っています。気軽にリプください、たくさん交流しましょう！', '代表取締役CEO及びCTOの井上顧基氏が率いるElithは、東京都文京区に本社を構え、最前線での技術革新を追求しています。 詳細情報. 本シンポジウムの詳細や ...', '参加無料！「今、知っておきたい！生成AIエージェントの世界」など講演や展示を2024年3月14日（木）～3月15日（金）に開催。先着100名の入場となります。', 'e